In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [9]:
# First the sample_sheets and clinical data from the two batches need to be merged. This will likely already be done
path = '../data/metadata/'
file = 'gdc_samplesheet.csv'
if os.path.exists(path+file):
    print('Smaplesheet has already been created and therefore the clinical data as well')
else:
    batch_1_sheet = pd.read_csv('../data/metadata/batch1/gdc_sample_sheet.2023-02-21.tsv', sep='\t', header = 0)
    batch_2_sheet = pd.read_csv('../data/metadata/batch2/gdc_sample_sheet.2023-02-21.tsv', sep='\t', header = 0)
    batch_1_clinical = pd.read_csv('../data/metadata/batch1/clinical.tsv', sep='\t', header = 0)
    batch_2_clinical = pd.read_csv('../data/metadata/batch2/clinical.tsv', sep='\t', header = 0)
    sample_sheet = pd.concat([batch_1_sheet,batch_2_sheet])
    clinical = pd.concat([batch_1_clinical,batch_2_clinical])
    
    sample_sheet.to_csv(path+file, header = True, sep = ',', index = None)
    clinical.to_csv(path+'clinical.csv', header = True, sep = ',', index = None)
    
    #Check if everthing is there (from missing_files we know there should be 11582 files in case for the sample sheet)
    print(len(batch_1_sheet)+len(batch_2_sheet))
    print(len(sample_sheet))
    print(len(batch_1_clinical)+len(batch_2_clinical))
    print(len(clinical))

11582
11582
21934
21934


In [2]:
HRD_scores = pd.read_csv('../data/HRD_scores_pan_cancer.csv', sep=',', header = 0)
file_names = HRD_scores['File Name']
case_ids = [s.split('.')[1] for s in file_names]
HRD_scores.insert(1, "case_id", case_ids)
HRD_scores.to_csv('../data/HRD_scores_pan_cancer_case_id.csv', sep=',', header = True, index = None)

In [8]:
sample_sheet = pd.read_csv('../data/metadata/gdc_samplesheet.csv', sep=',', header = 0)
clinical =  pd.read_csv('../data/metadata/clinical.csv', sep=',', header = 0)

HRD_scores = pd.read_csv('../data/HRD_scores_pan_cancer_case_id.csv', sep=',', header = 0)

# print(HRD_scores)
# for colname in list(clinical.columns):
#     print(colname+ ': '+ str(clinical.loc[0].at[colname]))
# # print(list(sample_sheet.columns))
# for colname in list(sample_sheet.columns):
#     print(colname+ ': '+ str(sample_sheet.loc[0].at[colname]))




case_ids_double = sample_sheet['Case ID']
case_ids = [s.split(',')[0] for s in case_ids_double]
sample_sheet = sample_sheet.assign(case_submitter_id=case_ids)


score_sample = pd.merge(HRD_scores, sample_sheet, on = 'File Name')


clinical.rename(columns={'project_id':'Project ID'}, inplace=True)
pid_primarydiag = clinical[['case_submitter_id','Project ID','primary_diagnosis']]

# print(pid_primarydiag[['case_submitter_id','Project ID']].value_counts(ascending = True))

pid_primarydiag_unique = pid_primarydiag.drop_duplicates(
  subset = ['case_submitter_id','Project ID','primary_diagnosis'],
  keep = 'last').reset_index(drop = True)

# print(pid_primarydiag_unique[['case_submitter_id','Project ID','primary_diagnosis']].value_counts(ascending = True))

scsa_clinical = pd.merge(score_sample, pid_primarydiag_unique, how = 'left', on = ['case_submitter_id','Project ID'])

print(scsa_clinical)


df_missing = HRD_scores[~HRD_scores['File Name'].isin(scsa_clinical['File Name'])]
#print(len(df_missing))


print(sample_sheet['case_submitter_id'].isin(clinical['case_submitter_id']).value_counts())
print(HRD_scores['case_id'].isin(clinical['case_id']).value_counts())
print(HRD_scores['case_id'])
scsa_clinical.to_csv('../data/HRD_scores_pan_cancer_annotated.csv', sep=',', header = True, index = None)


/tmp/ipykernel_55279/2208490534.py:2: DtypeWarning: Columns (3,48) have mixed types. Specify dtype option on import or set low_memory=False.
  clinical =  pd.read_csv('../data/metadata/clinical.csv', sep=',', header = 0)


                                               File Name  \
0      TARGET-ALL-P2.0161a290-9c04-59dc-a9c0-e027319d...   
1      TARGET-ALL-P2.01de10c6-5408-5edf-8898-0a60475d...   
2      TARGET-ALL-P2.01e8e594-06a1-573a-b1a7-df45a9d4...   
3      TARGET-ALL-P2.01fcca6c-76ef-5a82-a89c-c0485522...   
4      TARGET-ALL-P2.02a31c57-90da-5e4f-92b4-19929c3b...   
...                                                  ...   
11577  TCGA-UVM.da85b309-bb4c-4c22-90d5-30b0aec5a3d7....   
11578  TCGA-UVM.e373293c-f908-4559-915d-f41344ee67b3....   
11579  TCGA-UVM.ea4e0ebd-aabe-413c-9cba-8f772d98f0af....   
11580  TCGA-UVM.ea579fa9-83b7-4193-b460-b73c85c1f679....   
11581  TCGA-UVM.ffeec777-6964-4409-b332-ec94736d49f1....   

                                    case_id  LOH  TAI  LST  HRD_sum  \
0      0161a290-9c04-59dc-a9c0-e027319d335d    2    2    4        8   
1      01de10c6-5408-5edf-8898-0a60475d2464    0    0    0        0   
2      01e8e594-06a1-573a-b1a7-df45a9d4cc0e    0    0    0        

In [85]:
print(sample_sheet['case_submitter_id'].isin(clinical['case_submitter_id']).value_counts())

True     11556
False       26
Name: case_submitter_id, dtype: int64


In [96]:
df_missing = score_sample[~score_sample['File Name'].isin(scsa_clinical['File Name'])]
print(len(HRD_scores))
print(len(sample_sheet))
print(len(clinical))
print(len(df_missing))
print(df_missing['File Name'])

11582
11582
21934
26
1693     TCGA-BRCA.ae2856f4-dafc-45b6-96af-6c6615cf9475...
2736     TCGA-COAD.b3e35145-c013-4465-b2d3-773db827f1f0...
2807     TCGA-COAD.dc06c8cc-003e-44e0-b566-6565273deb37...
3278     TCGA-GBM.42eeadf9-e914-47f0-adf5-f261e42df086....
3572     TCGA-GBM.d5053d9f-5ff5-4682-8df7-95284eea011a....
3580     TCGA-GBM.d76b9192-75e0-4882-a883-d69cad8e3e72....
5651     TCGA-LGG.a30789af-a78e-4512-bb2f-764b3bd8b688....
7892     TCGA-OV.e8242e31-e8eb-4cf6-81db-27f5be5294e4.a...
8801     TCGA-READ.15d71143-92b6-4ad6-87e1-328f45fb7bae...
10127    TCGA-TGCT.13cd303a-1d78-4457-b664-4190074c2609...
10139    TCGA-TGCT.230a12a1-316d-4488-b050-e492a5b1fa77...
10149    TCGA-TGCT.3803123b-39df-436e-8e84-20d0d6cbcaeb...
10152    TCGA-TGCT.3f6a09bf-af0f-42ca-913e-fb24bc0f3932...
10166    TCGA-TGCT.53cf153a-ce45-48c2-8175-ca58ba7d2a11...
10194    TCGA-TGCT.7bd03897-b4fc-4502-8d16-4d15ad94250f...
10206    TCGA-TGCT.8ac7972b-c0da-4e3a-bdda-8eae0cf2cef8...
10215    TCGA-TGCT.99b42dcd-2b52-49

In [97]:
df_missing.to_csv('../data/missing_subtype_files.csv', sep=',', header = True, index = None)

In [11]:
print(len(scsa_clinical[scsa_clinical['Project ID'] == 'TCGA-OV']))
print(len(sample_sheet[sample_sheet['Project ID'] == 'TCGA-OV']))

588
589


In [12]:
files = glob.glob('../data/allele_specific_cnv/allele_cnv_txt/*.seg.txt')
files = [os.path.basename(file) for file in files]

In [14]:
projects = [s.split('.')[0] for s in files]

In [19]:
print(len(projects))

11582


In [26]:
sum_ = 0
for project in np.unique(projects):
    print(project + ': ' + str(projects.count(project)))
    sum_ = sum_ +  projects.count(project)
print(sum_)

TARGET-ALL-P2: 294
TARGET-AML: 92
TARGET-CCSK: 11
TARGET-OS: 81
TCGA-ACC: 90
TCGA-BLCA: 396
TCGA-BRCA: 1084
TCGA-CESC: 301
TCGA-CHOL: 36
TCGA-COAD: 509
TCGA-DLBC: 48
TCGA-ESCA: 184
TCGA-GBM: 542
TCGA-HNSC: 521
TCGA-KICH: 66
TCGA-KIRC: 574
TCGA-KIRP: 298
TCGA-LAML: 190
TCGA-LGG: 527
TCGA-LIHC: 373
TCGA-LUAD: 544
TCGA-LUSC: 514
TCGA-MESO: 86
TCGA-OV: 589
TCGA-PAAD: 182
TCGA-PCPG: 167
TCGA-PRAD: 491
TCGA-READ: 172
TCGA-SARC: 254
TCGA-SKCM: 468
TCGA-STAD: 436
TCGA-TGCT: 156
TCGA-THCA: 505
TCGA-THYM: 123
TCGA-UCEC: 545
TCGA-UCS: 53
TCGA-UVM: 80
11582


In [17]:
print(HRD_scores[HRD_scores['case_id'].duplicated() == True])
print(clinical[clinical['case_id'].duplicated() == True])

Empty DataFrame
Columns: [File Name, case_id, LOH, TAI, LST, HRD_sum]
Index: []


In [15]:
sample_type = list(sample_sheet['Sample Type'])
sum_ = 0
sample_list = list()
for type_ in np.unique(sample_type):
    words = type_.split(',')
    sample_list.append(type_)
    if 'Metastatic' in words:
        sum_ = sum_ + 1
    print(type_ + ': ' + str(sample_type.count(type_)))
print('Metastatic types: '+str(sum_))

Additional - New Primary, Blood Derived Normal: 5
Blood Derived Normal, Additional - New Primary: 5
Blood Derived Normal, Metastatic: 191
Blood Derived Normal, Primary Blood Derived Cancer - Bone Marrow: 114
Blood Derived Normal, Primary Blood Derived Cancer - Peripheral Blood: 9
Blood Derived Normal, Primary Tumor: 4445
Blood Derived Normal, Recurrent Blood Derived Cancer - Bone Marrow: 3
Blood Derived Normal, Recurrent Tumor: 32
Bone Marrow Normal, Primary Blood Derived Cancer - Bone Marrow: 59
Bone Marrow Normal, Primary Blood Derived Cancer - Peripheral Blood: 5
Bone Marrow Normal, Primary Tumor: 3
Bone Marrow Normal, Recurrent Blood Derived Cancer - Bone Marrow: 7
Buccal Cell Normal, Primary Tumor: 1
Metastatic, Blood Derived Normal: 195
Metastatic, Solid Tissue Normal: 2
Primary Blood Derived Cancer - Bone Marrow, Blood Derived Normal: 101
Primary Blood Derived Cancer - Bone Marrow, Bone Marrow Normal: 54
Primary Blood Derived Cancer - Peripheral Blood, Blood Derived Normal: 11
P

In [13]:
code_table = pd.read_csv('../data/TCGA_code_tables/diseaseStudy.tsv', sep='\t', header = 0)
code_table['Project ID'] = 'TCGA-'+ code_table['Study Abbreviation']


target_rows = pd.DataFrame({'Study Abbreviation': ['ALL-P2','AML','CCSK','OS'],
                            'Study Name': ['Acute Lymphoblastic Leukemia - Phase II','Acute Myeloid Leukemia','Clear Cell Sarcoma of the Kidney','Osteosarcoma'],
                            'Project ID': ['TARGET-ALL-P2','TARGET-AML','TARGET-CCSK','TARGET-OS']})
code_table = pd.concat([code_table, target_rows]).reset_index(drop=True)
code_table.to_csv('../data/TCGA_code_tables/diseaseStudy_updated.csv', sep=',', header = True, index = None)

In [9]:
sampletype_mapping = {
    'Additional - New Primary, Blood Derived Normal': 'Additional - New Primary, Blood Derived Normal',
    'Blood Derived Normal, Additional - New Primary': 'Additional - New Primary, Blood Derived Normal',
    'Blood Derived Normal, Metastatic': 'Metastatic, Blood Derived Normal',
    'Blood Derived Normal, Primary Blood Derived Cancer - Bone Marrow': 'Primary Blood Derived Cancer - Bone Marrow, Blood Derived Normal',
    'Blood Derived Normal, Primary Blood Derived Cancer - Peripheral Blood': 'Primary Blood Derived Cancer - Peripheral Blood, Blood Derived Normal',
    'Blood Derived Normal, Primary Tumor': 'Primary Tumor, Blood Derived Normal',
    'Blood Derived Normal, Recurrent Blood Derived Cancer - Bone Marrow': 'Recurrent Blood Derived Cancer - Bone Marrow, Blood Derived Normal',
    'Blood Derived Normal, Recurrent Tumor': 'Recurrent Tumor, Blood Derived Normal',
    'Bone Marrow Normal, Primary Blood Derived Cancer - Bone Marrow': 'Primary Blood Derived Cancer - Bone Marrow, Bone Marrow Normal',
    'Bone Marrow Normal, Primary Blood Derived Cancer - Peripheral Blood': 'Primary Blood Derived Cancer - Peripheral Blood, Bone Marrow Normal',
    'Bone Marrow Normal, Primary Tumor': 'Primary Tumor, Bone Marrow Normal',
    'Bone Marrow Normal, Recurrent Blood Derived Cancer - Bone Marrow': 'Recurrent Blood Derived Cancer - Bone Marrow, Bone Marrow Normal',
    'Buccal Cell Normal, Primary Tumor':'Primary Tumor, Buccal Cell Normal',
    'Metastatic, Blood Derived Normal': 'Metastatic, Blood Derived Normal',
    'Metastatic, Solid Tissue Normal': 'Metastatic, Solid Tissue Normal',
    'Primary Blood Derived Cancer - Bone Marrow, Blood Derived Normal': 'Primary Blood Derived Cancer - Bone Marrow, Blood Derived Normal',
    'Primary Blood Derived Cancer - Bone Marrow, Bone Marrow Normal': 'Primary Blood Derived Cancer - Bone Marrow, Bone Marrow Normal',
    'Primary Blood Derived Cancer - Peripheral Blood, Blood Derived Normal': 'Primary Blood Derived Cancer - Peripheral Blood, Blood Derived Normal',
    'Primary Blood Derived Cancer - Peripheral Blood, Bone Marrow Normal': 'Primary Blood Derived Cancer - Peripheral Blood, Bone Marrow Normal',
    'Primary Blood Derived Cancer - Peripheral Blood, Solid Tissue Normal' : 'Primary Blood Derived Cancer - Peripheral Blood, Solid Tissue Normal',
    'Primary Tumor, Blood Derived Normal' : 'Primary Tumor, Blood Derived Normal',
    'Primary Tumor, Bone Marrow Normal': 'Primary Tumor, Bone Marrow Normal',
    'Primary Tumor, Buccal Cell Normal': 'Primary Tumor, Buccal Cell Normal',
    'Primary Tumor, Solid Tissue Normal': 'Primary Tumor, Solid Tissue Normal',
    'Recurrent Blood Derived Cancer - Bone Marrow, Bone Marrow Normal': 'Recurrent Blood Derived Cancer - Bone Marrow, Bone Marrow Normal',
    'Recurrent Tumor, Blood Derived Normal': 'Recurrent Tumor, Blood Derived Normal',
    'Recurrent Tumor, Solid Tissue Normal':'Recurrent Tumor, Solid Tissue Normal',
    'Solid Tissue Normal, Metastatic': 'Metastatic, Solid Tissue Normal',
    'Solid Tissue Normal, Primary Blood Derived Cancer - Peripheral Blood': 'Primary Blood Derived Cancer - Peripheral Blood, Solid Tissue Normal',
    'Solid Tissue Normal, Primary Tumor': 'Primary Tumor, Solid Tissue Normal',
    'Solid Tissue Normal, Recurrent Tumor': 'Recurrent Tumor, Solid Tissue Normal' 
}

type_mapping = {
    'Additional - New Primary, Blood Derived Normal' : 'Primary',
    'Metastatic, Blood Derived Normal': 'Metastatic',
    'Metastatic, Solid Tissue Normal' : 'Metastatic',
    'Primary Blood Derived Cancer - Bone Marrow, Blood Derived Normal': 'Primary',
    'Primary Blood Derived Cancer - Bone Marrow, Bone Marrow Normal' : 'Primary',
    'Primary Blood Derived Cancer - Peripheral Blood, Blood Derived Normal' : 'Primary',
    'Primary Blood Derived Cancer - Peripheral Blood, Bone Marrow Normal' : 'Primary',
    'Primary Blood Derived Cancer - Peripheral Blood, Solid Tissue Normal' : 'Primary',
    'Primary Tumor, Blood Derived Normal' : 'Primary',
    'Primary Tumor, Bone Marrow Normal': 'Primary',
    'Primary Tumor, Buccal Cell Normal': 'Primary',
    'Primary Tumor, Solid Tissue Normal': 'Primary',
    'Recurrent Blood Derived Cancer - Bone Marrow, Bone Marrow Normal': 'Recurrent',
    'Recurrent Tumor, Blood Derived Normal': 'Recurrent',
    'Recurrent Tumor, Solid Tissue Normal':'Recurrent',
    'Recurrent Blood Derived Cancer - Bone Marrow, Blood Derived Normal': 'Recurrent'
}

In [10]:
scsa_clinical['sampleType_correct'] = ""

for sampletype in sampletype_mapping:
    scsa_clinical.loc[scsa_clinical['Sample Type'] == sampletype,'sampleType_correct'] = sampletype_mapping[sampletype]

scsa_clinical['Type'] = ""

for type_ in type_mapping:
    scsa_clinical.loc[scsa_clinical['sampleType_correct'] == type_,'Type'] = type_mapping[type_]

scsa_clinical['PID_Type'] = scsa_clinical['Project ID'] + ' ' + scsa_clinical['Type']


code_table = pd.read_csv('../data/TCGA_code_tables/diseaseStudy_updated.csv', sep=',', header = 0)
code_table = code_table[['Study Name','Project ID']]

scsa_clinical = pd.merge(scsa_clinical, code_table, on = ['Project ID'])

print(scsa_clinical)

scsa_clinical.to_csv('../data/HRD_scores_pan_cancer_annotated_typecorrect.csv', sep=',', header = True, index = None)

                                               File Name  \
0      TARGET-ALL-P2.0161a290-9c04-59dc-a9c0-e027319d...   
1      TARGET-ALL-P2.01de10c6-5408-5edf-8898-0a60475d...   
2      TARGET-ALL-P2.01e8e594-06a1-573a-b1a7-df45a9d4...   
3      TARGET-ALL-P2.01fcca6c-76ef-5a82-a89c-c0485522...   
4      TARGET-ALL-P2.02a31c57-90da-5e4f-92b4-19929c3b...   
...                                                  ...   
11577  TCGA-UVM.da85b309-bb4c-4c22-90d5-30b0aec5a3d7....   
11578  TCGA-UVM.e373293c-f908-4559-915d-f41344ee67b3....   
11579  TCGA-UVM.ea4e0ebd-aabe-413c-9cba-8f772d98f0af....   
11580  TCGA-UVM.ea579fa9-83b7-4193-b460-b73c85c1f679....   
11581  TCGA-UVM.ffeec777-6964-4409-b332-ec94736d49f1....   

                                    case_id  LOH  TAI  LST  HRD_sum  \
0      0161a290-9c04-59dc-a9c0-e027319d335d    2    2    4        8   
1      01de10c6-5408-5edf-8898-0a60475d2464    0    0    0        0   
2      01e8e594-06a1-573a-b1a7-df45a9d4cc0e    0    0    0        

In [2]:
gdc_cases = [1067,561,536,517,517,510,509,507,498,493,486,466,454,436,391,369,299,293,284,248,190,183,181,167,162,150,123,90,86,81,80,73,66,53,48,36,11]
files_numbers = [1084,589,545,521,574,542,527,544,505,514,491,468,509,436,396,373,301,294,298,254,190,184,182,172,167,156,123,90,86,81,80,92,66,53,48,36,11]
diff = list()
for i,num in enumerate(gdc_cases):
    diff.append(abs(num-files_numbers[i]))
print(sum(diff))

361
